## Horses Or Humans

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/1I2aWD5twJdrq0SI2QD91HB5zCvvvCE_i">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

# Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

# The Horses Or Humans Dataset

This dataset contains 500 CGI images of horses and 500 CGI images of humans.

Horse (class 0)
Human (class 1)

We want to train a machine learning model to classify examples as either horse or human based on the images.
![title](horses_or_humans.png)

## Loading

In [ ]:
(ds_train, ds_test), ds_info = tfds.load('horses_or_humans', split=['train', 'test'], shuffle_files=True, as_supervised=True, with_info =True)

In [ ]:
fig = tfds.show_examples(ds_train, ds_info, rows=4, cols=4)
print(ds_info)

## Preparing

In [ ]:
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64
ds_train = ds_train.map(normalize_img, num_parallel_calls = AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls = AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.prefetch(AUTOTUNE)

## Training

In [ ]:
model = keras.Sequential(
    [
        keras.Input((300, 300, 3)),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Flatten(),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)

model.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

history = model.fit(ds_train, epochs=5, verbose=2)

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

## Evaluation

In [ ]:
model.evaluate(ds_test)